# 优化器类

我们通过张量封装了梯度计算链路，通过层封装了前向传播链路。

现在，我们将引入**优化器**（Optimizer）类来封装反向传播链路，根据梯度更新参数。

-----------------

至此，我们完成了对神经网络模型三条数据链路的封装：
* **张量**：梯度计算链路，组成计算图；
* **层**：前向传播链路；
* **损失函数**：前向传播链路和梯度计算链路的连接点；
* **优化器**：反向传播链路。

In [17]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [18]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return len(self.data)

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础层

为了辅助实现反向传播链路的自动化，我们需要在层中添加一个新的属性：**参数列表**（parameters），用来返回本层需要参与反向传播的张量（通常是本层的模型参数）。

In [19]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [20]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

**基础优化器**（Base Optimizer）是一个抽象类，定义了一个**参数更新函数**（step）的虚拟接口，用来根据参数列表中所有张量的梯度，更新它们的数值。

创建一个优化器，我们需要知道：

* **参数列表**（parameters）：各层参与反向传播的张量列表，这些张量都需要根据梯度进行更新；
* **学习率**（lr）：梯度更新比例系数。

In [21]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    @abstractmethod
    def step(self):
        pass

## 数据

### 特征、标签

In [22]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

在线性层，我们把权重和偏置加入**参数列表**（parameters）。它们需要根据梯度进行更新。

In [23]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad * x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 损失函数（平均平方差）

In [24]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

我们的第一个优化器类是**随机梯度下降优化器**（SGDOptimizer），按照固定的**学习率**（lr）更新**参数列表**（parameters）中的所有张量。

In [25]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

## 设置

### 学习率

In [26]:
LEARNING_RATE = 0.00001

## 验证

### 建模

In [27]:
layer = Linear(feature.size, 1)
print(layer)

loss_fn = MSELoss()
# 优化器
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)

Linear[weight(1, 2); bias(1,)]


### 推理

In [28]:
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: Tensor([43.05])


### 评估

In [29]:
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(14871.802500000002)


## 训练

### 梯度计算

In [30]:
loss.backward()

### 反向传播

完成梯度计算以后，我们可以调用优化器的**更新参数函数**（step），完成一次迭代的反向传播。

简单地讲，梯度计算完成以后，梯度值已经保存在各张量的**梯度**（grad）中。反向传播时，优化器只需要遍历**参数列表**（parameters），根据保存的梯度值，更新各张量的数值。

In [31]:
optimizer.step()

### 重新评估

In [32]:
prediction = layer(feature)
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(12503.020514375934)


通过优化器类，我们实现了反向传播链路（更新参数）的自动化。

## 课后练习

**父节点列表**（parents）和**参数列表**（parameters）的设计，使得计算图的遍历和反向传播都可以自动化完成。尝试把不同的张量从父节点列表和参数列表中移除，观察反向传播的结果如何？